In [1]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from parse_logs.parse_base import LogDir
from parse_logs.parse_coordinator_logs import read_design

from parse_logs.parse_coordinator_logs import get_flat_design
from flatten_json import flatten

from verification.verify_experiment import check_experiment

sns.set(rc={"figure.figsize":(16, 8)})
# parser.root_data_path = '../tests/data'
#trial_run_objects = parser.get_trial_runs(
#    exp_run_code='experiment_2021_04_01_16_20_00_run_1')

LogDir.base_data_path = '../data'
check_mc, check_te = check_experiment('experiment_2021_07_29_15_33_17_run_1')



les-02/aaaaab> last task executed was "approach_arm".
les-02/aaaaap> last task executed was "wait_lab_arm_to_complete_pick_up_sample".
les-02/aaaabb> last task executed was "authenticate_nurse".
les-02/aaaabp> last task executed was "wait_lab_arm_to_complete_pick_up_sample".
les-02/aaaacb> last task executed was "authenticate_nurse".
les-02/aaaacp> last task executed was "wait_lab_arm_to_complete_pick_up_sample".
les-02/aaabab> last task executed was "wait_lab_arm_to_complete_pick_up_sample".
les-02/aaabap> last task executed was "wait_lab_arm_to_complete_pick_up_sample".
les-02/aaabbb> last task executed was "wait_lab_arm_to_complete_pick_up_sample".
les-02/aaabbp> last task executed was "wait_lab_arm_to_complete_pick_up_sample".
les-02/aaabcb> last task executed was "wait_lab_arm_to_complete_pick_up_sample".
les-02/aaabcp> last task executed was "wait_lab_arm_to_complete_pick_up_sample".
les-02/aaacab> last task executed was "approach_arm".
parsing ending task turtlebot6:navto_lab, b

In [2]:
cmc_df = pd.DataFrame.from_records(check_mc)
cte_df = pd.DataFrame.from_records(check_te)

In [3]:
error_cmc_df = cmc_df[cmc_df.result == False]
error_cmc_df

,trial,property,result


In [4]:
#cte_df_res = cte_df.groupby(["property", "result"])
cte_group_by_result = cte_df.groupby(["property", "result"])["result"].size().to_frame().rename(columns = {'result': 'count'}).reset_index()#.pivot(columns='result')
cte_group_by_result

,property,result,count
0,assigned_robots_received_local_mission,False,53
1,assigned_robots_received_local_mission,True,1243
2,tasks_were_executed_in_order,False,67
3,tasks_were_executed_in_order,True,1229


In [5]:
y = pd.pivot_table(cte_group_by_result,  values='count', index=['property'], columns=['result'] ).reset_index().rename(columns={True:'passed', False:'npassed'})
y

result,property,npassed,passed
0,assigned_robots_received_local_mission,53,1243
1,tasks_were_executed_in_order,67,1229


In [6]:
y.assign(reason=lambda row:row.passed /(row.passed + row.npassed)*100 )

result,property,npassed,passed,reason
0,assigned_robots_received_local_mission,53,1243,95.910494
1,tasks_were_executed_in_order,67,1229,94.830247


In [7]:
cte_group_by_result


,property,result,count
0,assigned_robots_received_local_mission,False,53
1,assigned_robots_received_local_mission,True,1243
2,tasks_were_executed_in_order,False,67
3,tasks_were_executed_in_order,True,1229


In [8]:
error_cte_df = cte_df[cte_df.result == False].reset_index().sort_values(['exec_group', 'trial'])
with pd.option_context('display.max_rows', 120, 'display.max_columns', 10):
    display(error_cte_df)

,index,exec_group,trial,property,result
93,2086,les-01,abaccp,assigned_robots_received_local_mission,False
94,2087,les-01,abaccp,tasks_were_executed_in_order,False
95,2089,les-01,abbaab,tasks_were_executed_in_order,False
96,2144,les-01,abcbcb,assigned_robots_received_local_mission,False
97,2145,les-01,abcbcb,tasks_were_executed_in_order,False
98,2172,les-01,acabab,assigned_robots_received_local_mission,False
99,2173,les-01,acabab,tasks_were_executed_in_order,False
100,2211,les-01,acbbap,tasks_were_executed_in_order,False
101,2252,les-01,accbcb,assigned_robots_received_local_mission,False
102,2253,les-01,accbcb,tasks_were_executed_in_order,False
